En primer lugar instalamos y cargamos las librerías necesarias.

In [1]:
from pytube import YouTube #Para descargar los videos
from pydub import AudioSegment
import pydub #Para trabajar con el audio

from huggingsound import SpeechRecognitionModel #Para cargar el modelo
from pocketsphinx import AudioFile, get_model_path
import os

import re #Para la parte de normalizar
import string 
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from autocorrect import Speller #Para los errores ortográficos
spell = Speller(lang='es')
from jiwer import wer #Para la métrica

import time #para medir el tiempo

[nltk_data] Downloading package punkt to /home/jonathan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jonathan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Creamos ahora las funciones que se encargan de hacer todo el proceso de transcripción y normalización.

In [2]:
def cargarVideo(video, formato):
    video.streams.filter(only_audio=True)
    audio = video.streams.get_by_itag(22)
    mp4 = audio.download()
    audio = AudioSegment.from_file(mp4)
    audio.export('video.'+str(formato), format=formato)
    return audio

In [3]:
def fragmentos(audio, formato):
    audio_length = len(audio) # Lo mide en milisegundos
    
    chunk_counter = 1
    
    point = 60000 # Duración de cada trozo
    rem = 8000 # Tiempo de solapación
    flag = 0
    start = 0
    end = 0
    
    lista = {} # Me creo un diccionario donde ir almacenando las rutas con sus chunk
    
    for i in range(0, 2*audio_length, point):
        if i == 0:
            start = 0
            end = point
        else:
            start = end-rem
            end = start+point
        if end >= audio_length:
            end = audio_length
            flag = 1 # Para que pare
        chunk = audio[start:end] # Nos quedamos con la parte del audio
        
        chunk_name = f'chunk_{chunk_counter}' # Le damos un nombre distinto a cada chunk
        
        chunk.export(chunk_name, format=formato) # Guardamos el chunk
        lista[chunk_name+'.'+str(formato)] = chunk # Me guardo el chunk en la lista
        chunk_counter += 1
        if flag == 1:
            break
    return lista

In [4]:
def wav2vec(lista):
    wav2vec = SpeechRecognitionModel("jonatasgrosman/wav2vec2-large-xlsr-53-spanish")
    audio_paths = []
    transcripciones = []
    for i in range(1, len(lista)+1):
        audio_paths.append(['chunk_'+str(i)])
        transcripciones.append(wav2vec.transcribe(audio_paths[i-1]))
    return audio_paths, transcripciones

In [5]:
def pocketsphinx(lista):
    model_path = get_model_path()
    config = {
        'verbose': False,
        'hmm': os.path.join(model_path, 'es-es'), # Carpeta del modelo acústico
        'lm': os.path.join(model_path, 'es-20k.lm.bin'), #Modelo de lenguaje
        'dict': os.path.join(model_path, 'es.dict')
    }
    audio_paths = []
    transcripciones = []
    for i in range(1, len(lista)+1):
        texto = ''
        audio_paths.append('chunk_'+str(i))
        audio = AudioFile(**config, audio_file = audio_paths[i-1])
        for frase in audio:
            texto = texto + str(frase) + ' '
        transcripciones.append(texto)
    return audio_paths, transcripciones

In [6]:
def concatenarWav2vec(transcripcion):
    texto = transcripcion[0][0]['transcription']
    for i in range(1, len(transcripcion)):
        a = len(transcripcion[i][0]['transcription'])
        j = 1 # Empiezo en 1 porque 0 será el espacio
        esta = False
        long = len(texto)
        esp = texto[::-1].index(' ')
        texto = texto[:long-esp] # Corto a partir del último espacio por si la palabra está cortada
        esp2 = transcripcion[i][0]['transcription'].index(' ')
        transcripcion[i][0]['transcription'] = transcripcion[i][0]['transcription'][esp2:] # Corto a partir del primer espacio por si la palabra está cortada
        while esta == False and j<a:
            if transcripcion[i][0]['transcription'][1:j] in texto[:]:
                j = j+1
            else:
                texto = texto + transcripcion[i][0]['transcription'][j:]
                esta = True
    return texto

In [7]:
def concatenarPocketsphinx(transcripcion):
    texto = transcripcion[0]
    for i in range(1, len(transcripcion)):
        a = len(transcripcion[i])
        j = 1 # Empiezo por 1 porque 0 será espacio
        esta = False
        long = len(texto)
        esp = texto[::-1].index(' ')
        texto = texto[:long-esp] # Corto a partir del último espacio por si la palabra está cortada
        esp2 = transcripcion[i].index(' ')
        transcripcion[i] = transcripcion[i][esp2:] # Corto a partir del primer espacio por si la palabra está cortada
        while esta == False and j<a:
            if transcripcion[i][1:j] in texto[:]:
                j = j+1
            else:
                texto = texto + transcripcion[i][j:]
                esta = True
    return texto

In [8]:
def numero_to_letras(numero):
	indicador = [("",""),("mil","mil"),("millon","millones"),("mil","mil"),("billon","billones")]
	entero = int(numero)
	decimal = int(round((numero - entero)*100))
 
	contador = 0
	numero_letras = ""
	while entero >0:
		a = entero % 1000
		if contador == 0:
			en_letras = convierte_cifra(a,1).strip()
		else :
			en_letras = convierte_cifra(a,0).strip()
		if a==0:
			numero_letras = en_letras+" "+numero_letras
		elif a==1:
			if contador in (1,3):
				numero_letras = indicador[contador][0]+" "+numero_letras
			else:
				numero_letras = en_letras+" "+indicador[contador][0]+" "+numero_letras
		else:
			numero_letras = en_letras+" "+indicador[contador][1]+" "+numero_letras
		numero_letras = numero_letras.strip()
		contador = contador + 1
		entero = int(entero / 1000)
	
	if decimal == 0:
		numero_letras = numero_letras
	else:
		numero_letras = numero_letras+" con " + str(decimal) +"/100"

	return numero_letras
 
 
def convierte_cifra(numero,sw):
	lista_centana = ["",("cien","ciento"),"doscientos","trescientos","cuatrocientos","quinientos","seiscientos","setecientos","ochocientos","novecientos"]
	lista_decena = ["",("diez","once","doce","trece","catorce","quince","dieciseis","diecisiete","dieciocho","diecinueve"),
					("veinte","veinti"),("treinta","treinta y "),("cuarenta" , "cuarenta y "),
					("cincuenta" , "cincuenta y "),("sesenta" , "sesenta y "),
					("setenta" , "setenta y "),("ochenta" , "ochenta y "),
					("noventa" , "noventa y ")
				]
	lista_unidad = ["",("un" , "uno"),"dos","tres","cuatro","cinco","seis","siete","ocho","nueve"]
	centena = int (numero / 100)
	decena = int((numero -(centena * 100))/10)
	unidad = int(numero - (centena * 100 + decena * 10))
 
	texto_centena = ""
	texto_decena = ""
	texto_unidad = ""
 
	#Validad las centenas
	texto_centena = lista_centana[centena]
	if centena == 1:
		if (decena + unidad)!=0:
			texto_centena = texto_centena[1]
		else :
	 		texto_centena = texto_centena[0]
 
	#Valida las decenas
	texto_decena = lista_decena[decena]
	if decena == 1 :
		texto_decena = texto_decena[unidad]
	elif decena > 1 :
		if unidad != 0 :
			texto_decena = texto_decena[1]
		else:
			texto_decena = texto_decena[0]
 	#Validar las unidades
    
	if decena != 1:
 		texto_unidad = lista_unidad[unidad]
 		if unidad == 1:
 			texto_unidad = texto_unidad[sw]
	return "%s %s %s" %(texto_centena,texto_decena,texto_unidad)

In [9]:
def normalizar(texto):
    tokens=word_tokenize(texto) #Separa el texto por palabras
    
    texto = [w.lower() for w in tokens] #Pasamos las palabras a minúsculas
    
    texto = [texto[i].replace(' apdo.', ' apartado').replace(' art. ',' articulo ').replace(' atte.',' atentamente').replace(' avda.',' avenida').replace(' cap.',' capítulo').replace(' cía.',' compañía').replace(' coord.',' coordinador').replace(' d.',' don').replace(' dña.',' doña').replace(' dcho.',' derecho').replace(' dcha.',' derecha').replace(' depto.',' departamento').replace(' dr.',' doctor').replace(' dra.',' doctora').replace(' etc.',' etcétera').replace(' fdo.',' firmado').replace(' izqdo.',' izquierdo').replace(' izqda.',' izquierda').replace(' max.',' máximo').replace(' min.',' mínimo').replace(' núm.',' número').replace(' pág.',' página').replace(' ej.',' ejemplo').replace(' prov.',' provincia').replace(' sr.',' señor').replace(' sra.',' señora').replace(' srta.',' señorita').replace(' tfno.',' teléfono') for i in range(len(texto))] #quitamos las abreviaciones
    for i in range(len(texto)):
        if texto[i].isdigit() and (texto[i] not in string.punctuation):
            texto[i]=numero_to_letras(int(texto[i]))
            tokens2=word_tokenize(texto[i])
            for j in range(len(tokens2)-1):
                texto.append('a') #me añado esto simplemente para no tener problemas con el rango
            for j in range(i+1, len(texto)-3):
                texto[len(texto)-j+i]=texto[len(texto)-j+i-len(tokens2)+1]
            for j in range(i, i+len(tokens2)-1):
                texto[j+len(tokens2)-1] = texto[j]
            for j in range(i, i+len(tokens2)):
                texto[j]=tokens2[j-i]
    tokens=[w.lower() for w in texto] #Pasamos las palabras a minúsculas por si había números
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    stripped = [re_punc.sub('',w) for w in tokens]
    stripped=[stripped[i].replace('¿', '').replace('¡','').replace("'",'') for i in range(len(tokens))] #quita los símbolos de puntuación que string.punctuation no tiene en cuenta
    words = [word for word in stripped if word.isalpha()] #Elimina los signos de puntuación
    return words

In [10]:
def ortografia(texto):
    for word in texto:
        word = spell(word)
    return texto

In [11]:
def unir(texto):
    texto_norm = ""
    for i in range(len(texto)):
        texto_norm = texto_norm + ' ' + texto[i]
    return texto_norm

In [12]:
def transcripcion(video, modelo, formato):
    inicio = time.time() # Para calcular el tiempo que tarda en ejecutarse
    audio = cargarVideo(video, formato)
    lista = fragmentos(audio, formato)
    if modelo==wav2vec:
        audio_paths, transcripciones = wav2vec(lista)
        texto = concatenarWav2vec(transcripciones)
    elif modelo==pocketsphinx:
        audio_paths, transcripciones = pocketsphinx(lista)
        texto = concatenarPocketsphinx(transcripciones)
    else:
        print('El modelo elegido no funciona.')
    normalizado = normalizar(texto)
    ortog = ortografia(normalizado)
    texto_norm = unir(ortog)
    fin = time.time()
    tiempo = fin-inicio
    return texto_norm, tiempo

Cargamos el vídeo y le aplicamos la función.

In [13]:
video = YouTube('https://www.youtube.com/watch?v=YwMyCV6UzRU')
texto, tiempo = transcripcion(video, wav2vec, 'wav')
print(tiempo)

11/16/2022 13:11:12 - INFO - huggingsound.speech_recognition.model - Loading model...


100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:34<00:00, 34.62s/it]


306.5403685569763


In [14]:
texto, tiempo = transcripcion(video, pocketsphinx, 'raw')
print(tiempo)

522.1310942173004
